In [ ]:
import os
import csv
import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, array_to_img, ImageDataGenerator
import preprocess
import imageio
import itertools

In [ ]:
files = preprocess.get_files("data", verbose=False)
X, Y = preprocess.load_imgs(files[0:1])
X = X[0]
Y = Y[0]
preprocess.show_img(X, Y)
print(X.shape, Y.shape)

In [ ]:
def square_img_split_idcs(img_size=2084, partition_size=512):
    assert img_size >= partition_size, "Cannot upsample img!"
    # number of partions on each axis
    partitions = int(np.ceil(img_size / partition_size))
    overhang = partitions * partition_size - img_size
    overlap = int(np.floor(overhang / partitions))
    stride = partition_size - overlap
    
    x_start = 0
    y_start = 0
    x_end = partition_size
    y_end = partition_size
    
    idcs = []
    for ix in range(partitions - 1):
        idcs.append(slice(x_start, x_end, 1))
        x_start += stride
        x_end += stride
    x_end = img_size
    x_start = x_end - partition_size
    idcs.append(slice(x_start, x_end, 1))
    
    idcs = list(itertools.product(idcs, repeat=2))
    
    assert len(idcs) == partitions**2
    return idcs

def split_and_store_img(X, Y, split_idcs, path, file_form, verbose=True):
    file_form = file_form + "{:d}_{}.png"
    i = 0
    X_assembled = (np.random.random(X.shape) * 255).astype(X.dtype)  
    Y_assembled = (np.random.random(Y.shape) * 255).astype(Y.dtype)
    for idx in split_idcs:
        X_part = X[idx]
        Y_part = Y[idx]
        X_assembled[idx] = X_part
        Y_assembled[idx] = Y_part
        
        if np.sum(Y_part) > 0:
            path_x = os.path.join(path, file_form.format(i, "x"))
            path_y = os.path.join(path, file_form.format(i, "y"))
            
            imageio.imwrite(path_x, X_part)
            imageio.imwrite(path_y, Y_part)
            
            X_r = imageio.imread(path_x)
            Y_r = imageio.imread(path_y)
            
            assert np.sum(X_part - X_r) == 0, "X does not match written version!"
            assert np.sum(Y_part - Y_r) == 0, "Y does not match written version!"
            i += 1
    assert np.sum(X - X_assembled) == 0, "X does not match assembled version!"
    assert np.sum(Y - Y_assembled) == 0, "Y does not match assembled version!"

In [ ]:
idcs = square_img_split_idcs(2084, 512)
split_and_store_img(X, Y, idcs, "augmented/")

In [ ]:
preprocess.show_ndimg(X, Y)
preprocess.show_ndimg(X)
preprocess.show_ndimg(Y)
x = None
y = None
for idx in idcs:
    print(idx)
    X_sub = X[idx]
    Y_sub = Y[idx]
    if np.sum(Y_sub) > 0:
        preprocess.show_ndimg(X_sub, Y_sub)
        x = X_sub
        y = Y_sub

In [ ]:
x.astype(np.uint8)

In [ ]:
imageio.imwrite('X_sub.png', x)
imageio.imwrite('Y_sub.png', y)

In [ ]:
preprocess.show_ndimg(y, y_r)

In [ ]:
x_r = imageio.imread("X_sub.png")
y_r = imageio.imread("Y_sub.png")
y_r = y_r.reshape((512, 512, 1)) / 255

In [ ]:
print(np.sum(y_r))
print(y_r.shape)
print(y_r.dtype)

In [ ]:
print(np.sum(y))
print(y.shape)
print(y.dtype)

In [ ]:
X.shape

In [ ]:
idx = (slice(0, 512), slice(0, 512))

In [ ]:
X[idx].shape

In [ ]:
img_size = 512
sub_imgs = []
slides = int(np.ceil(2084 / img_size))
extra = slides * img_size - 2084
overlap = int(np.floor(extra / slides))
stride = img_size - overlap

idcs = []
x_start = 0
x_end = 512
y_start = 0
y_end = 512
for ix in range(slides - 1):
    y_start = 0
    y_end = 512
    for iy in range(slides - 1):
        print(x_start, x_end, y_start, y_end)
        idcs.append((x_start, x_end, y_start, y_end))
        y_start += stride
        y_end += stride
    y_end = 2084
    y_start = y_end - 512
    print(x_start, x_end, y_start, y_end)
    idcs.append((x_start, x_end, y_start, y_end))
    x_start += stride
    x_end += stride
x_end = 2084
x_start = x_end - 512
y_start = 0
y_end = 512
for _ in range(slides - 1):
    print(x_start, x_end, y_start, y_end)
    idcs.append((x_start, x_end, y_start, y_end))
    y_start += stride
    y_end += stride
y_end = 2084
y_start = y_end - 512
print(x_start, x_end, y_start, y_end)
idcs.append((x_start, x_end, y_start, y_end))
    
        #x_sub = X[ix * img_size : (ix + 1) * img_size, iy * img_size : (iy + 1) * img_size, :]
        #y_sub = Y[ix * img_size : (ix + 1) * img_size, iy * img_size : (iy + 1) * img_size, :]
        #preprocess.show_ndimg(x_sub, y_sub)

In [ ]:
files = preprocess.get_files("data", verbose=False)
X, Y = preprocess.load_imgs(files)

In [ ]:
idcs = square_img_split_idcs(2084, 512)
for i in range(X.shape[0]):
    split_and_store_img(X[i], Y[i], idcs, "data/split/", file_form=str(i) + "_")

In [ ]:
[i]